### Inputs and Imports

In [2]:
import re
from collections import Counter

input_file_list = []
sample_input_file_list = []

day = '05'

with open(f'Inputs\\day_{day}.txt', 'r') as input_file: 
    for line in input_file.readlines(): 
        input_file_list.append(line.rstrip('\n'))
with open(f'Inputs\\day_{day}_sample.txt', 'r') as input_file: 
    for line in input_file.readlines(): 
        sample_input_file_list.append(line.rstrip('\n'))        

########################        
# Part One Sample Answer: 5 
########################
# Part Two Sample Answer: 12
########################

### Part One

--- Day 5: Hydrothermal Venture ---


You come across a field of hydrothermal vents on the ocean floor! These vents constantly produce large, opaque clouds, so it would be best to avoid them if possible.

They tend to form in lines; the submarine helpfully produces a list of nearby lines of vents (your puzzle input) for you to review. For example:
```
0,9 -> 5,9
8,0 -> 0,8
9,4 -> 3,4
2,2 -> 2,1
7,0 -> 7,4
6,4 -> 2,0
0,9 -> 2,9
3,4 -> 1,4
0,0 -> 8,8
5,5 -> 8,2
```
Each line of vents is given as a line segment in the format x1,y1 -> x2,y2 where x1,y1 are the coordinates of one end the line segment and x2,y2 are the coordinates of the other end. These line segments include the points at both ends. In other words:

- An entry like 1,1 -> 1,3 covers points 1,1, 1,2, and 1,3.
- An entry like 9,7 -> 7,7 covers points 9,7, 8,7, and 7,7.  
For now, only consider horizontal and vertical lines: lines where either x1 = x2 or y1 = y2.

So, the horizontal and vertical lines from the above list would produce the following diagram:
```
.......1..
..1....1..
..1....1..
.......1..
.112111211
..........
..........
..........
..........
222111....
```
In this diagram, the top left corner is 0,0 and the bottom right corner is 9,9. Each position is shown as the number of lines which cover that point or . if no line covers that point. The top-left pair of 1s, for example, comes from 2,2 -> 2,1; the very bottom row is formed by the overlapping lines 0,9 -> 5,9 and 0,9 -> 2,9.

To avoid the most dangerous areas, you need to determine the number of points where at least two lines overlap. In the above example, this is anywhere in the diagram with a 2 or larger - a total of 5 points.

Consider only horizontal and vertical lines. At how many points do at least two lines overlap?

In [6]:
input_file_list

['491,392 -> 34,392',
 '337,52 -> 485,52',
 '256,605 -> 256,959',
 '889,142 -> 153,878',
 '189,59 -> 512,382',
 '399,193 -> 598,193',
 '578,370 -> 795,153',
 '79,450 -> 569,450',
 '565,444 -> 270,149',
 '39,28 -> 39,846',
 '114,353 -> 114,383',
 '356,61 -> 356,327',
 '140,132 -> 515,132',
 '361,848 -> 361,527',
 '466,257 -> 466,784',
 '818,397 -> 818,14',
 '693,554 -> 693,984',
 '171,290 -> 171,655',
 '989,889 -> 170,70',
 '527,855 -> 527,549',
 '209,355 -> 486,355',
 '800,430 -> 291,939',
 '980,38 -> 31,987',
 '964,559 -> 964,799',
 '491,612 -> 930,173',
 '57,977 -> 958,76',
 '149,465 -> 349,465',
 '512,624 -> 629,507',
 '460,943 -> 460,441',
 '988,29 -> 988,968',
 '104,337 -> 441,337',
 '939,48 -> 939,546',
 '941,904 -> 498,461',
 '850,972 -> 649,771',
 '840,901 -> 23,84',
 '231,790 -> 231,873',
 '230,668 -> 840,58',
 '410,922 -> 435,897',
 '341,337 -> 341,406',
 '264,752 -> 258,752',
 '457,969 -> 457,757',
 '465,42 -> 465,350',
 '748,783 -> 502,783',
 '461,930 -> 461,142',
 '392,265

In [7]:
# vent_lines_raw = sample_input_file_list
vent_lines_raw = input_file_list

# function to convert the input file into a list of 4 numbers:
# the start and end coordinates for x and y (x1, y1, x2, y2)
def convert_vent_lines(vent_lines_raw):
    print('-- converting vent lines input')
    new_vent_lines = []
    for i in vent_lines_raw:
        # split the string using ' -> ' and ',' and converting each element to an integer
        new_line_format = [int(x) for x in re.split(" -> |,", i)]
        new_vent_lines.append(new_line_format)
    return new_vent_lines
    
# create a list of each point a vent line crosses in a string format of 'x-y'
def get_vent_line_points(vent_lines):
    print('-- creating list of points a vent line crosses')
    vent_line_points = []
    for line in vent_lines:       
        x1, y1, x2, y2 = line # assigning each element of the line to its x/y variable
        
        # create a list of all points between x1, y1 and x2, y2
        # if the x values match create a range from the min/max y values and iterate over that
        if x1 == x2:
            pass
            y_range = range(min(y1, y2), max(y1, y2)+1)
            vent_line_points.extend([f'{x1}-{y}' for y in y_range])
            
        # if the y values match create a range from the min/max x values and iterate over that
        elif y1 == y2:
            pass
            x_range = range(min(x1, x2), max(x1, x2)+1)
            vent_line_points.extend([f'{x}-{y1}' for x in x_range])
            
        # otherwise it's a diagonal direction so pass (for now....)
        else:
            pass
    return vent_line_points   

# takes the list of all vent points all lines cross and outputs only the points crossed 2 or more times
def count_vent_points(vent_points):
    # turn the vent points into a counter/collections object with a count of each occurence
    counted_vent_points = Counter(vent_points)
    # turn the above into a new dict for only those points with a count >= 2
    vent_points_2_or_more = {key: value for key, value in counted_vent_points.items() if value >= 2}
    print(f'\nThere are {len(vent_points_2_or_more)} grid points where the vent lines have crossed over 2 or more times')
    return vent_points_2_or_more
    
vent_lines            = convert_vent_lines(vent_lines_raw)   
vent_points           = get_vent_line_points(vent_lines)
vent_points_2_or_more = count_vent_points(vent_points)
vent_points_2_or_more

-- converting vent lines input
-- creating list of points a vent line crosses

There are 6856 grid points where the vent lines have crossed over 2 or more times


{'39-392': 2,
 '48-392': 2,
 '61-392': 2,
 '75-392': 2,
 '97-392': 2,
 '120-392': 2,
 '142-392': 2,
 '171-392': 2,
 '189-392': 2,
 '217-392': 2,
 '230-392': 2,
 '254-392': 2,
 '278-392': 2,
 '286-392': 2,
 '293-392': 3,
 '295-392': 2,
 '318-392': 2,
 '328-392': 2,
 '330-392': 2,
 '337-392': 2,
 '341-392': 2,
 '344-392': 2,
 '370-392': 2,
 '378-392': 2,
 '405-392': 2,
 '417-392': 2,
 '420-392': 2,
 '437-392': 2,
 '461-392': 2,
 '466-392': 2,
 '468-392': 2,
 '470-392': 2,
 '487-392': 2,
 '422-52': 2,
 '429-52': 2,
 '465-52': 2,
 '256-627': 2,
 '256-664': 2,
 '256-669': 2,
 '256-681': 2,
 '256-683': 2,
 '256-689': 2,
 '256-723': 2,
 '256-738': 2,
 '256-757': 2,
 '256-770': 2,
 '256-836': 2,
 '256-859': 2,
 '256-866': 2,
 '256-868': 2,
 '256-942': 2,
 '256-947': 2,
 '256-951': 2,
 '256-955': 2,
 '405-193': 2,
 '422-193': 2,
 '429-193': 2,
 '461-193': 2,
 '465-193': 2,
 '470-193': 2,
 '487-193': 2,
 '513-193': 2,
 '536-193': 2,
 '542-193': 2,
 '554-193': 2,
 '578-193': 2,
 '598-193': 2,
 '9

### Part Two

Unfortunately, considering only horizontal and vertical lines doesn't give you the full picture; you need to also consider diagonal lines.

Because of the limits of the hydrothermal vent mapping system, the lines in your list will only ever be horizontal, vertical, or a diagonal line at exactly 45 degrees. In other words:

- An entry like 1,1 -> 3,3 covers points 1,1, 2,2, and 3,3.
- An entry like 9,7 -> 7,9 covers points 9,7, 8,8, and 7,9.

Considering all lines from the above example would now produce the following diagram:
```
  0123456789
  ----------
0|1.1....11.
1|.111...2..
2|..2.1.111.
3|...1.2.2..
4|.112313211
5|...1.2....
6|..1...1...
7|.1.....1..
8|1.......1.
9|222111....
```
You still need to determine the number of points where at least two lines overlap. In the above example, this is still anywhere in the diagram with a 2 or larger - now a total of 12 points.

Consider all of the lines. At how many points do at least two lines overlap?

In [7]:
# adding logic to add the line points of a diagonal line to the get_vent_line_points funtion 

# vent_lines_raw = sample_input_file_list
vent_lines_raw = input_file_list

# create a list of each point a vent line crosses in a string format of 'x-y'
def get_vent_line_points_v2(vent_lines):
    print('-- creating list of points a vent line crosses')
    vent_line_points = []
    for line in vent_lines:
        # assigning each element of the line to its x/y variable
        x1, y1, x2, y2 = line
        
        # create a list of all points between x1, y1 and x2, y2
        # vertical line - the x values match so create a range from the min/max y values and iterate over that
        if x1 == x2:
            pass
            y_range = range(min(y1, y2), max(y1, y2)+1)
            vent_line_points.extend([f'{x1}-{y}' for y in y_range])
            
        # horizontal line - the y values match so create a range from the min/max x values and iterate over that
        elif y1 == y2:
            pass
            x_range = range(min(x1, x2), max(x1, x2)+1)
            vent_line_points.extend([f'{x}-{y1}' for x in x_range])
            
        # diagonal - do something different based on the diagonal direction
        else:
            # calculate length of line then convert to range to iterate over
            line_len_range = range((max(x1, x2) - min(x1, x2))+1)
            
            # ⬊ top left to bottom right, add to both x and y values
            if   x1 < x2 and y1 < y2:
                for index in line_len_range:
                    new_x, new_y = x1+(index), y1+(index)
                    vent_line_points.append(f'{new_x}-{new_y}')
                    
            # ⬉ bottom right to top left, minus from both x and y values
            elif x1 > x2 and y1 > y2:
                for index in line_len_range:
                    new_x, new_y = x1-(index), y1-(index)
                    vent_line_points.append(f'{new_x}-{new_y}')
            
            # ⬈ bottom left to top right, add to x and minus from y
            elif x1 < x2 and y1 > y2:
                for index in line_len_range:
                    new_x, new_y = x1+(index), y1-(index)
                    vent_line_points.append(f'{new_x}-{new_y}')
            
            # ⬋ top right to bottom left, minus from x and add to y
            elif x1 > x2 and y1 < y2:
                for index in line_len_range:
                    new_x, new_y = x1-(index), y1+(index)
                    vent_line_points.append(f'{new_x}-{new_y}')
                
    return vent_line_points   

vent_lines            = convert_vent_lines(vent_lines_raw)   
vent_points           = get_vent_line_points_v2(vent_lines)
vent_points_2_or_more = count_vent_points(vent_points)

-- converting vent lines input
-- creating list of points a vent line crosses

There are 20666 grid points where the vent lines have crossed over 2 or more times


### Other Methods 

In [ ]:
######################################################################
# I originally used the below functions to create all possible grid points 
# this was based on the max x and y values within the vent lines
# then I iterated through this to count the amount fo times it appeared in the vent points
# finally I created a list from this with all the grid points with a count of 2 or more

# when it came to implementing this it took ages iterate through and count if a point was in the vent point list
# instead I opted to create the grid points dict from only those points in the vent point list
######################################################################

# create a dictionary of each possible grid point from the vent list, with a 0 counter as the value
def get_grid_points(vent_lines): 
    print('-- creating dictionary for all possible grid points')
    # create list of all x and y coordinates:
    x_points, y_points = [], []
    for line in vent_lines:
        for coordinate in line[:2]:
            x_points.append(coordinate)
        for coordinate in line[2:]:
            y_points.append(coordinate)    

    # create x and y ranges (starting at 0)
    x_range, y_range = range(max(x_points)+1), range(max(y_points)+1)
    
    # turn these ranges into a list of all possible grid points in a string format of 'x-y'
    grid_points = [f'{x}-{y}' for x in x_range for y in y_range]
    # convert to a dictionary
    grid_point_dict = dict.fromkeys(grid_points, 0)
    return grid_point_dict

# update all the records within the grid_points with the count of their appearance in vent_points
def count_vent_points_in_grid_points(vent_points,grid_points):
    print('-- updating grid point dict with count of all points a vent line crosses')
    for key, value in grid_points.items():
        updated_record = {key:vent_points.count(key)}
        grid_points_dict.update(updated_record)
        print(f'grid points dict entry for {key} = {grid_points[key]}')
    return grid_points_dict

# takes the list of all grid points and returns the volumne of points with a line count of 2 or over
def count_points_over_2(grid_points_dict):
    print('-- creating new list of all grid points that have been crossed over 2 or more times')
    points_over_2 = []
    for key, value in grid_points_dict.items():
        if value >= 2:
            points_over_2.append([key, value])
    print(f'\nthere are {len(points_over_2)} grid points where the vent lines have crossed over 2 or more times')
    return points_over_2  

# updated_grid_points_dict = count_vent_points_in_grid_points(vent_points,grid_points_dict)
# points_over_2 = count_points_over_2(updated_grid_points_dict)